
# NanoLab Examples
This notbook provides some code examples of how to build laboratory workflows using the NanoLab engine.

**Remember** The aim is not to express a perfect scientific protocol. But to build an intiuition of how samples and metadata about samples flow through a series of common activities in the lab.

## Basic Usage
In this section we will introduce the different unit operations and their usage

### Preliminaries 

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import nanolab as lab
import matplotlib.pyplot as plt 

### Dispensing Samples

In [4]:
my_dispenser = lab.Dispenser()

#Dispense a gram of Sodium Cloride
tube1 = my_dispenser("Sodium Cloride", 1, "g")
print(tube1)

#Dispense a gram of Sodium Cloride at 0.9% concentration
tube2 = my_dispenser("Sodium Cloride solution", 50, "ml", concentration=0.9, conc_unit="%", solvent="Water")
print(tube2)


1 g: Sodium Cloride
50 ml: Sodium Cloride solution at 0.9% in Water


### Registering Samples

Either rename a sample, or prepred a string onto the current name

In [5]:

#Totally rename a sample    
rename1 = lab.Register("Salt")
print(rename1(tube1))

#Prepend and identify onto the id of a sample
rename2 = lab.Register("Salt_", prepend=True)
print(rename2(tube2))



1 g: Salt
50 ml: Salt_Sodium Cloride solution at 0.9% in Water


### Exploring Geneology

Using the printHistory() method of Sample to see all of the operations that happened to a sample since it was introduced into the nanolab.

In [6]:
tube1.contents.printHistory()

Sample:Salt <-Unit Op(Register)
  Sample:Sodium Cloride <-Unit Op(Dispenser)


In [7]:
tube2.contents.printHistory()

Sample:Salt_Sodium Cloride solution at 0.9% in Water <-Unit Op(Register)
  Sample:Sodium Cloride solution at 0.9% in Water <-Unit Op(Dispenser)


### Serialising Subsamples

Append a serial number to the end of each sample id

In [9]:
# create a group of samples
sample_array = [my_dispenser("Ethanol", 10, "ml") for _ in range(5)]
print("Before: ", sample_array)

#Serialise each samples
serialize = lab.Serialiser()
print("After: ", list(map(serialize, sample_array)))

#print geneology
print("\nGeneology")
sample_array[1].contents.printHistory()

Before:  [10 ml: Ethanol, 10 ml: Ethanol, 10 ml: Ethanol, 10 ml: Ethanol, 10 ml: Ethanol]
After:  [10 ml: Ethanol_0, 10 ml: Ethanol_1, 10 ml: Ethanol_2, 10 ml: Ethanol_3, 10 ml: Ethanol_4]

Geneology
Sample:Ethanol_1 <-Unit Op(Serialiser)
  Sample:Ethanol <-Unit Op(Dispenser)


### Incubating Samples

Holding a sample at a specific time and temperature

In [10]:
#
incubate = lab.Incubator(30, "min", 37, "c")

#Rename after incubation
rnai = lab.Register("Incubated_", True)

tube3 = rnai(incubate(my_dispenser("Sodium Cloride", 1, "g")))
print(tube3)
print()
tube3.contents.printHistory()


1 g: Incubated_Sodium Cloride

Sample:Incubated_Sodium Cloride <-Unit Op(Register)
  Sample:Sodium Cloride <-Unit Op(Incubated)
    Sample:Sodium Cloride <-Unit Op(Dispenser)


### Aliquoting samples

Note here labware becomes a feature as volumes are tracked at this levels

In [22]:
aliquoter = lab.Aliquoter(10, "ml")

tube4 = my_dispenser("Sodium Cloride solution", 50, "ml")
print(f"Before: {tube4}")

aliquot1 = aliquoter(tube4)
print("After:")
print(f"Original: {tube4}")
print(f"Aliquot: {aliquot1}")



Before: 50 ml: Sodium Cloride solution
After:
Original: 40 ml: Sodium Cloride solution
Aliquot: 10 ml: Sodium Cloride solution


In [25]:
# Other cases

#Wrong Units
tube4a = my_dispenser("Sodium Cloride solution", 50, "gallons")
try:
    print(aliquoter(tube4a))
except ValueError as e:
    print("Error:", e)

#Too much sample requested
aliquoter.quantity = 60
try:
    print()
    print("Aliquot tube:", aliquoter(tube4a))
except ValueError as e:
    print("Error:", e)


aliquoter.force = True
try:
    print()
    print("Aliquot tube:", aliquoter(tube4a))
    print("Origional tube:",tube4a)
except ValueError as e:
    print("Error:", e)

#When forced the origional tube forgets the quantity it has.
#But you can create infinite sample in the aliquot! 


60 ml: Sodium Cloride solution

Aliquot tube: 60 ml: Sodium Cloride solution

Aliquot tube: 60 ml: Sodium Cloride solution
Origional tube: Sodium Cloride solution


### Mixing of Samples

Here we start to Mix our samples. We can only add mix in one sample at a time. 

If the labware quantities are the same unit then they will be added

In [28]:
mix_in = lab.Mixer()

tube5a =  my_dispenser("Sodium Cloride", 1, "g")
tube5b = my_dispenser("Water", 50, "ml")

tube5 = mix_in(tube5a,tube5b)
print(tube5)

tube5.contents.printHistory()






Mixture
Sample:Mixture <-Unit Op(Mixer)
  Sample:Sodium Cloride <-Unit Op(Dispenser)
  Sample:Water <-Unit Op(Dispenser)


## Scenarios

### Bake Cookies
This is one of my favourite goto examples about how to think about unit operations in the lab and other patterns.

The goal is to bake some cookies using our laboratory proceedures based on the following recipie

Ingredients
* 125g butter, softened
* 175g light brown soft sugar
* 60g caster sugar
* 1 large egg
* 225g plain flour
* ½ tsp baking powder
* ½ tsp bicarbonate of soda
* ½ tsp sea salt flakes
* 275g dark cooking chocolate, chopped

Method
1. Beat the butter and sugars with an electric whisk for 5 mins or until pale and fluffy. Whisk in the egg.

2. In a separate bowl, mix the flour, baking powder, bicarbonate of soda and sea salt. Add to the butter mixture and stir until just combined. 

3. Stir in the chocolate until just incorporated and there are no white streaks in the dough. 

4. Preheat the oven to gas 4, 180°C, fan 160°C and line 3 large baking trays with nonstick baking paper. 

5. Divide the dough into 14 equal-size balls and transfer to the trays, leaving lots of space between them. Flatten slightly with the heel of your hand to make pucks 5-6cm wide.

6. Bake for 13-15 mins until lightly golden but still a little pale in the centre; they’ll firm up as they cool.

7. Cool on the trays for 5 mins before transferring to a wire rack to cool completely. 


In [ ]:
dispense = lab.Dispenser()
mix = lab.Mixer()
rename = lab.Register("Cookie Batter")
make_cookie = lab.Aliquoter(1, "Cookie")
number = lab.Serialiser()
bake = lab.Incubator(15,"min", "180", "c")
cool = lab.Incubator(30,"min","room temp", "")

print(dispense)
print(mix)
print(rename)
print(make_cookie)
print(number)
print(bake)
print(cool)

In [ ]:

#collect ingrediants
butter = dispense("Butter", 125, "g")
castor_sugar = dispense("Caster Suger", 60,"g")
brown_sugar = dispense("Brown Sugar", 175,"g")
egg = dispense("Egg",1 ,"Large")
vanilla = dispense("Vanilla Extract",1 ,"tsp")
flour = dispense("Plain Flour",225 ,"g")
bicarb = dispense("Bicarbonate of Soda", "1/2" ,"tsp")
baking_powder = dispense("Baking Powder", "1/2" ,"tsp")
sea_salt = dispense("Sea Salt Flakes", "1/2" ,"tsp")
choc_chips = dispense("Mixed Cocolate Chips (White, Milk and Plain)",275 ,"g")

ingredients = (butter, castor_sugar, brown_sugar, egg, vanilla,flour,bicarb,baking_powder,sea_salt,choc_chips)


In [ ]:

#Mixing Ingrediants per recipie


buttermix = mix(mix(butter, brown_sugar),castor_sugar)
print(buttermix)

batter = mix(mix(mix(flour, baking_powder),bicarb),sea_salt)
print(batter)

batter = mix(batter, buttermix)
print(batter)

batter = mix(batter, choc_chips)
print(batter)







In [ ]:

cookie_batter = rename(batter)
cookies = [cool(bake(number(make_cookie(batter)))) for i in range(15)]
print(cookies)



In [ ]:
print(cookies[0].contents.parents)